In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("data/UCI_Credit_Card.csv")

### About Dataset

This dataset contains information on **default payments** of credit card clients in Taiwan covering a six month period from April 2005 to September 2005. It includes columns with insights about the clients' demographic characteristics, payment histories, and credit behavior

##### Demographic Variables
- `USER_ID`: Unique identifier for each client.

- `CREDIT_LIMIT`: Credit limit in NT dollars (includes individual and family/supplementary credit).

- `SEX`: Gender of the client (1 = Male, 2 = Female).

- `EDUCATION`: Educational background of the client:
  - 1 = Graduate School
  - 2 = University
  - 3 = High School
  - 4 = Others

- `MARITALSTATUS`: Marital status of the client (1 = Married, 2 = Single, 3 = Others).

- `AGE`: Age of the client in years.


##### Billing and Payment Variables
- `BILL_AMTx`: Amount of bill statement (NT dollar)
  - `BILL_AMT1` - `BILL_AMT6`: Amount of bill statement in September - April, 2005


- `PAYMENTDELAY_1` to `PAYMENTDELAY_6`: Monthly repayment status from August 2005 to April 2005 (reverse chronological order).
  - `PAYMENTDELAY_x`: Repayment status where
    - -1 = Payment made on time
    - 1 = Payment delayed for 1 month
    - 2 = Payment delayed for 2 months
    - …
    - 8 = Delay of 8 months
    - 9 = Delay of 9 months or more

- `PAID_AMTx`: Amount of previous payment (NT dollar)
  - `PAID_AMT1` - `PAID_AMT6`: Amount of previous payment in September - April, 2005

##### Target Variable
- `DID_DEFAULT_PAYMENT`: Whether the client defaulted on payment in the next month (1 = Yes, 0 = No)

In [ ]:
df.head()

In [ ]:
df.info()

### Training
Since the dataset is about credit defaults, we will train a classifier that can predict whether a client will default on their payment in the next month. The performance of the model will be evaluated using the Accuracy and F1 Score of the model

In [6]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [7]:
# Segregate the data into features and target variables

target_col = []
feature_cols = []


X = []
y = []

In [8]:
# Split the data into training and test sets

X_train, X_test, y_train, y_test = ([], [], [], [])

In [ ]:
param_grid = {
    'max_depth': np.arange(3, 10),
    'criterion' : ['gini','entropy'],
    'max_leaf_nodes': [5,10,20,100],
    'min_samples_split': [2, 5, 10, 20]
}

# Create the grid
grid_tree = GridSearchCV(DecisionTreeClassifier(), param_grid, cv = 5, scoring= 'accuracy')

# Training
grid_tree.fit(X_train, y_train)


print(f"Best Estimator: {grid_tree.best_estimator_}")
print(f"Best Accuracy: {np.abs(grid_tree.best_score_)}")

### Model Export
Now that the classifier is trained, the next step is to deploy it within a Python service. To do this, we will export the trained model as a pickle file. By saving it in pickle format, we will simplify our inference by enabling quick model loading and reuse without having to do any complex setup process.

In [16]:
import joblib

In [ ]:
joblib.dump(grid_tree.best_estimator_, "risk_radar_model")

In [18]:
m = joblib.load("risk_radar_model")

In [19]:
single_entry = X_test.sample(n=1)

In [ ]:
prediction = m.predict(single_entry).item()
probs = m.predict_proba(single_entry)
confidence = probs.max(axis=1).item()

print(f"Prediction: {prediction}, Confidence: {confidence}")

In [ ]:
single_entry.to_dict(orient='records')